In [ ]:
import importlib
import sys
from urllib.request import urlretrieve
import time

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='latest', drake_build='continuous')

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Setup rendering (with xvfb), if necessary:
import os
if 'google.colab' in sys.modules and os.getenv("DISPLAY") is None:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start two meshcat servers (one for 2d) to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc_2d, zmq_url_2d, web_url_2d = start_zmq_server_as_subprocess(server_args=server_args)
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Imports
import numpy as np
from IPython.display import display, HTML
from ipywidgets import Textarea

from pydrake.all import ( 
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, 
    DiagramBuilder, RigidTransform, RotationMatrix, Box,    
    CoulombFriction, FindResourceOrThrow, FixedOffsetFrame, 
    GeometryInstance, MeshcatContactVisualizer, Parser, PlanarJoint,  
    JointIndex, Simulator, ProcessModelDirectives, LoadModelDirectives
)

from functools import partial
import open3d as o3d
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

from pydrake.all import (
    ConnectPlanarSceneGraphVisualizer,
    ConnectDrakeVisualizer, DepthCameraProperties, RgbdSensor,
    RandomGenerator, UniformlyRandomRotationMatrix, RollPitchYaw,
    MakeRenderEngineVtk, RenderEngineVtkParams, Role, UnitInertia, set_log_level
)

from ipywidgets import Dropdown, FloatSlider, Layout
from pydrake.all import (
    Sphere, Cylinder, Box, Capsule, Ellipsoid, SpatialInertia)

import pydrake

#from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
from manipulation.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
from manipulation.meshcat_utils import draw_open3d_point_cloud, draw_points
from manipulation.open3d_utils import create_open3d_point_cloud
from manipulation.mustard_depth_camera_example import MustardExampleSystem
from manipulation.scenarios import AddRgbdSensors
from manipulation.utils import FindResource

set_log_level("warn");

ycb = [("cracker", "003_cracker_box.sdf"), 
        ("sugar", "004_sugar_box.sdf"), 
        ("soup", "005_tomato_soup_can.sdf"), 
        ("mustard", "006_mustard_bottle.sdf"), 
        ("gelatin", "009_gelatin_box.sdf"), 
        ("meat", "010_potted_meat_can.sdf")]


Cloning into '/opt/manipulation'...

Already on 'master'

/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK

ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.5.3 which is incompatible.
ERROR: nbclient 0.5.1 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.






In [ ]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torchvision.models
import collections
import math
import torch.nn.functional as F


from google.colab import drive
drive.mount('/content/gdrive')

SAMPLES = -1
BATCH_SIZE = 10
EPOCHS = 5
IMG_SHAPE = (480, 640, 4)
POSE_SHAPE = (6,)
VAL_PROPORTION = .1
device = "cuda"

Mounted at /content/gdrive


In [12]:
# cd to relevant directory

In [ ]:
# Define dataloader
class Dataset(torch.utils.data.Dataset):
    def __init__(self, num_items, dir_ind, val=False):
        self.x_files = []
        self.y_files = []
        for data_file_index in dir_ind:
            if val:
                x_dir = os.path.join(DATA_DIRECTORY, "xval" + str(data_file_index))
                y_dir = os.path.join(DATA_DIRECTORY, "yval" + str(data_file_index))
            else:
                x_dir = os.path.join(DATA_DIRECTORY, "x" + str(data_file_index))
                y_dir = os.path.join(DATA_DIRECTORY, "y" + str(data_file_index))
            for index in range(num_items):
                self.x_files.append(os.path.join(x_dir, str(index) + "img.npy"))
                self.y_files.append(os.path.join(y_dir, str(index) + "pose.npy"))
        self.indices = np.arange(len(self.x_files), dtype=int)
    
    def __getitem__(self, index):
        ind = self.indices[index]
        img = np.load(self.x_files[ind])
        img = torch.tensor(np.transpose(img, (2, 0, 1))) / 255.
        pose = torch.tensor(np.load(self.y_files[ind]))
        pos, rot = pose[:3], pose[3:]
        theta = 2*torch.acos(rot[0] / torch.sqrt(rot[0]**2 + rot[-1]**2))
        rot = torch.tensor([theta])
        return (img, [pos, rot])

    def __len__(self):
        return len(self.indices)

In [ ]:
# Define neural network

def conv(in_channels, out_channels, kernel_size):
    padding = (kernel_size-1) // 2
    assert 2*padding == kernel_size-1, "parameters incorrect. kernel={}, padding={}".format(kernel_size, padding)
    return nn.Sequential(
          nn.Conv2d(in_channels,out_channels,kernel_size,stride=1,padding=padding,bias=False),
          nn.ReLU(inplace=True),
        )

def fc(in_channels, out_channels):
    return nn.Sequential(
        nn.Linear(in_channels, out_channels),
    )

class normalize(nn.Module):
    def __init__(self):
        super(normalize, self).__init__()
    def forward(self, x):
        return torch.clamp(x, 0, 2 * np.pi)


class PoseEstimatorNet(nn.Module):
    def __init__(self):
        super(PoseEstimatorNet, self).__init__()
        self.resnet = torchvision.models.resnet18()
        self.initial = conv(4, 3, 1) # in_channels = 4 for RGBAlpha
        self.fc1 = fc(1000, 256)
        self.fc2_position = fc(256, 3)
        self.fc2_rotation = fc(256, 1)
        self.normalize = normalize()
        self.relu = torch.nn.ReLU() 
    def forward(self, x):
        # Convolve to be acceptable size for resnet
        x = self.initial(x)
        # ResNet18
        x = self.resnet(x)
        # Multi-layer perceptron
        x = self.fc1(x)
        x = self.relu(x)
        x_pos = self.fc2_position(x)
        x_rot = self.fc2_rotation(x)
        x_rot = self.normalize(x_rot)

        return x_pos, x_rot

In [ ]:
def similarity_metric(img, actual):
    L1_dist = np.abs(img - actual).mean()
    return np.exp(-L1_dist)

def inverse_graphics(img, initial_pose, max_iter = 10000, var_pos = .001, var_ori = np.pi/32):
    """
    Run MCMC to refine initial pose guesses for all objects
    """
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0005)

    parser = Parser(plant)

    parser.AddModelFromFile(FindResourceOrThrow(
        "drake/examples/manipulation_station/models/bin.sdf"))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("bin_base"))


    rs = np.random.RandomState()  # this is for python
    generator = RandomGenerator(rs.randint(1000))  # this is for c++

    # Only generate a few objects for testing.
    for i in [2]:
        object_num = i
        sdf = FindResourceOrThrow("drake/manipulation/models/ycb/sdf/" + ycb[object_num][1])
        parser.AddModelFromFile(sdf, f"object{i}")

    plant.Finalize()

    renderer = "my_renderer"
    scene_graph.AddRenderer(
        renderer, MakeRenderEngineVtk(RenderEngineVtkParams()))
    properties = DepthCameraProperties(width=640,
                                       height=480,
                                       fov_y=np.pi / 4.0,
                                       renderer_name=renderer,
                                       z_near=0.1,
                                       z_far=10.0)
    camera = builder.AddSystem(
        RgbdSensor(parent_id=scene_graph.world_frame_id(),
                   X_PB=RigidTransform(
                       RollPitchYaw(np.pi, 0, np.pi/2.0),
                       [0, 0, .8]),
                   properties=properties,
                   show_window=False))
    camera.set_name("rgbd_sensor")
    builder.Connect(scene_graph.get_query_output_port(),
                    camera.query_object_input_port())
    builder.ExportOutput(camera.color_image_output_port(), "color_image")

    # Note: if you're running this on a local machine, then you can 
    # use drake_visualizer to see the simulation.  (It's too slow to 
    # show the meshes on meshcat).
    vis = ConnectDrakeVisualizer(
        builder, 
        scene_graph
    )
    

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    plant_context = plant.GetMyContextFromRoot(context)
    
    prev_pose = None
    prev_score = -np.infty
    best_pose = None
    best_score = -np.infty
    best_img = None

    z = 0.07
    for ind in range(max_iter):
        for body_index in plant.GetFloatingBaseBodies():
            if prev_pose is None:
                theta = float(initial_pose[-1][0])
                pos = np.array([initial_pose[0][0], initial_pose[0][1], z])
            else:
                theta = float(prev_pose[-1][0])
                pos = np.array([prev_pose[0][0], prev_pose[0][1], z])
                if np.random.rand() < .5:
                    delta_theta = np.random.normal(loc=0, scale=var_ori)
                    delta_position  = np.random.normal(loc=0, scale=var_pos, size=2) * 0
                else:
                    delta_theta = np.random.normal(loc=0, scale=var_ori) * 0
                    delta_position  = np.random.normal(loc=0, scale=var_pos, size=2)
                
                theta += delta_theta
                if theta < 0:
                    theta += 2 * np.pi
                pos = np.minimum(np.array([.12, .18, z]), np.maximum(np.array([-.12, -.18, z]), pos + np.array([delta_position[0], delta_position[1], 0])))
            
            tf = RigidTransform(
                rpy=RollPitchYaw(0, 0, theta), 
                p=pos
                )
            
            plant.SetFreeBodyPose(plant_context, 
                                        plant.get_body(body_index),
                                        tf)
            
        simulator.AdvanceTo(1.0 if running_as_notebook else 0.1)
        for body_index in plant.GetFloatingBaseBodies():
            tf = plant.GetFreeBodyPose(plant_context, plant.get_body(body_index))
        color_image = diagram.GetOutputPort("color_image").Eval(context).data / 255.
        if ind == 0:
            init_img = color_image

        curr_score = similarity_metric(color_image, img)
        if curr_score >= prev_score or np.random.rand() < curr_score / prev_score:
            #rpy = pydrake.common.eigen_geometry.RollPitchYaw_(tf.rotation().matrix())
            theta = np.arccos(tf.rotation().matrix()[0][0])#rpy[-1]
            translation = tf.translation()
            prev_pose = [translation, [theta]]
            prev_score = curr_score
            if curr_score > best_score:
                best_score = curr_score
                best_pose = prev_pose
                best_img = color_image
        else:
            pass
        
        if ind % (max_iter // 10) == 0 and ind != 0:
            print(str(100 * ind / max_iter) + "% complete" )

    # Make tensor
    pos, ori = best_pose
    best_pose = [torch.tensor([pos]), torch.tensor([ori])]

    return best_score, best_pose, best_img, init_img

        



In [ ]:
# Distance metrics
class Loss1(nn.Module):
    def __init__(self, alpha = 1):
        super(Loss1, self).__init__()

    def forward(self, y_pred, y_target):
        position_loss = torch.sqrt((y_target[0] - y_pred[0]).pow(2).sum(dim=1)).mean()
        delta = torch.abs(y_pred[1] - y_target[1])
        orientation_loss = torch.minimum(2*np.pi - delta, delta).mean()
        return position_loss, orientation_loss

In [ ]:
DATA_DIRECTORY = os.path.join(os.getcwd(), "data")

torch.autograd.set_detect_anomaly(True)
print("Initializing model")
model = PoseEstimatorNet()
#model.to(device)
#model.load_state_dict(torch.load(os.path.join(os.getcwd(), "checkpoints2", "epoch2loss0.04361401511535754"), map_location="cpu"))
model.load_state_dict(torch.load(os.path.join(os.getcwd(), "checkpoints2", "epoch2loss0.0037583035621693403"), map_location="cpu"))
#model.load_state_dict(torch.load(os.path.join(os.getcwd(), "checkpoints2", "epoch9loss0.006022169968406322"), map_location="cpu"))

model.eval()
test_loader = torch.utils.data.DataLoader(Dataset(50, [""], val=True), batch_size = 1)
dist_metric = Loss1()

# Plot stuff
#n_plots = 3 #len(test_loader)
#f = plt.figure(figsize=(480.*n_plots / 500., 640.*3 / 500.))
#fig, axs = plt.subplots(n_plots, 3, figsize=(640.*3 / 200., 480.*n_plots / 200.))

bad_pos = 0
not_bad = 0
avg_pos_error = 0
avg_ori_error = 0
avg_ref_pos_error = 0
avg_ref_ori_error = 0

for i, [img, label] in enumerate(test_loader):
    print("-------------------------------" + str(i) + "-------------------------------")
    TEMP = img
    np_img = np.transpose(np.array(img).squeeze(), axes=(1, 2, 0))
    pred = model(img)
    pos, ori = dist_metric(pred, label)
    print("Running MCMC...")
    # Process into usable form
    new_pred = [pred[0].squeeze(), [2*np.pi*np.random.rand()]]
    best_score, best_pose, best_img, init_img = inverse_graphics(np_img, new_pred, max_iter = 100, var_pos = .001, var_ori = np.pi/4)
    print("Done!")
    print("Actual:")
    print(label)
    print("Neural Net Predicted:")
    print(pred)
    print("Position distance: " + str(float(pos)))
    print("Orientation distance: " + str(float(ori)))
    ###
    ref_pos, ref_ori = dist_metric(best_pose, label)
    print("MCMC Refined:")
    print(best_pose)
    print("Position distance: " + str(float(ref_pos)))
    print("Orientation distance: " + str(float(ref_ori)))
    '''if i in [7, 8, 9]:
        axs[i-7, 0].imshow(np_img)
        axs[i-7, 0].set_xticklabels([])
        axs[i-7, 0].set_yticklabels([])

        axs[i-7, 1].imshow(init_img)
        axs[i-7, 1].set_xticklabels([])
        axs[i-7, 1].set_yticklabels([])
        
        axs[i-7, 2].imshow(best_img)
        axs[i-7, 2].set_xticklabels([])
        axs[i-7, 2].set_yticklabels([]) '''
    if ref_pos > .1:
        bad_pos += 1
    else:
        not_bad += 1
        avg_pos_error += pos
        avg_ori_error += ori
        avg_ref_pos_error += ref_pos
        avg_ref_ori_error += ref_ori
    #break    
''' cols = ['Observed Image', 'Neural Net Pose Render', 'MCMC Refined Pose Render']
rows = ['Run {}'.format(row) for row in ['A', 'B', 'C']]

for ax, col in zip(axs[0], cols):
    ax.set_title(col)

for ax, row in zip(axs[:,0], rows):
    ax.set_ylabel(row) '''

Initializing model
-------------------------------0-------------------------------
Running MCMC...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: DrakeDeprecationWarning: Deprecated; due to be removed after 2021-03-01. Please use DepthRenderCamera instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: DrakeDeprecationWarning: Constructs an RgbdSensor in the same way as the above overload, but
using the ``CameraProperties`` portion of ``properties`` for color
(and label) properties, and all of ``properties`` for depth
properties.  /
(Deprecated.)

Deprecated:
    CameraProperties are being deprecated. Please use the RenderCamera
    variant. This will be removed from Drake on or after 2021-03-01.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: DrakeDeprecationWarning: Extends a Diagram with the required components to interface with
drake_visualizer. This must be called *during* Diagram building and
uses the given ``builder`` to add relevant subsystems and connections.

This is a convenience method to simplify some common boilerplat

10.0% complete
20.0% complete
30.0% complete
40.0% complete
50.0% complete
60.0% complete
70.0% complete
80.0% complete
90.0% complete
Done!
Actual:
[tensor([[-0.0523, -0.1007,  0.0468]], dtype=torch.float64), tensor([[2.0746]], dtype=torch.float64)]
Neural Net Predicted:
(tensor([[-0.0472, -0.0951,  0.0704]], grad_fn=<AddmmBackward>), tensor([[0.7311]], grad_fn=<ClampBackward>))
Position distance: 0.024733228866046165
Orientation distance: 1.3435250510576444
MCMC Refined:
[tensor([[-0.0483, -0.0981,  0.0700]], dtype=torch.float64), tensor([[1.1070]], dtype=torch.float64)]
Position distance: 0.0236513285168966
Orientation distance: 0.9676121796079715
-------------------------------1-------------------------------
Running MCMC...
10.0% complete
20.0% complete
30.0% complete
40.0% complete
50.0% complete
60.0% complete
70.0% complete
80.0% complete
90.0% complete
Done!
Actual:
[tensor([[ 0.0916, -0.0541,  0.0700]], dtype=torch.float64), tensor([[2.7559]], dtype=torch.float64)]
Neural Net

" cols = ['Observed Image', 'Neural Net Pose Render', 'MCMC Refined Pose Render']\nrows = ['Run {}'.format(row) for row in ['A', 'B', 'C']]\n\nfor ax, col in zip(axs[0], cols):\n    ax.set_title(col)\n\nfor ax, row in zip(axs[:,0], rows):\n    ax.set_ylabel(row) "

In [ ]:
print(bad_pos)
print(np.array([avg_pos_error, avg_ori_error, avg_ref_pos_error, avg_ref_ori_error]) / float(not_bad))

8
[tensor(0.0055, dtype=torch.float64, grad_fn=<DivBackward0>)
 tensor(1.2668, dtype=torch.float64, grad_fn=<DivBackward0>)
 tensor(0.0079, dtype=torch.float64) tensor(0.3824, dtype=torch.float64)]
